In [1]:
from iroha_helper import *
from new_helper import *
from super_helper import *
from loguru import logger

In [2]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 2

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
# user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
# project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

#for the index system
index_path = "indexdir"
index = open_dir(index_path)


In order to star a new notebook from here, the only dependency is `project_id` from Notebook 3

In [3]:
import pickle

# Load variable_a from the file
with open("temp/project_id.pkl", "rb") as f:
    project_id = pickle.load(f)

logger.info(project_id)  # Outputs: Hello from Notebook A!


2025-01-14 22:19:36.483 | INFO     | __main__:<module>:7 - 60032@test


In [4]:
schema = get_schema() #super_helper.py

logger.info(schema)

processed_data = process_files(directory_path, project_id, schema) #new_helper.py
 
    

2025-01-14 22:19:36.488 | INFO     | __main__:<module>:3 - <Schema: ['abstract', 'created', 'creator', 'date', 'description', 'file_cid', 'format', 'full_text', 'language', 'metadata_cid', 'modified', 'project_id', 'publisher', 'subject', 'title']>
2025-01-14 22:19:36.572 | INFO     | iroha_helper:tracer:244 - 	Entering "create_contract"
2025-01-14 22:19:36.595 | INFO     | iroha_helper:create_contract:263 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-14 22:19:36.598 | INFO     | iroha_helper:create_contract:263 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 22:19:38.726 | INFO     | iroha_helper:create_contract:263 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 22:19:38.755 | INFO     | iroha_helper:create_contract:263 - ('COMMITTED', 5, 0)
2025-01-14 22:19:38.757 | INFO     | iroha_helper:tracer:246 - 	Leaving "create_contract"
2025-01-14 22:19:38.758 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-14 22:19:38.777 | INFO     | i

In [5]:
account_detail = get_account_detail(project_id)
logger.info(f"{project_id}, {account_detail}")


2025-01-14 22:19:41.914 | INFO     | __main__:<module>:2 - 60032@test, { "admin@test" : { "file_1" : "QmdiRawzVNUiB28ENKQ7WefeFLEJ1xMjsJjwtHL2jnJ9xW, QmTPq62ZwkveRj1XWMEi4CkGRwD3eJ4LAvDeu73RuRtCv3", "linked_user" : "jolly_carson@test", "project_metadata_cid" : "QmUeLA7dzwBXgBhMu3xZu2cvMSM6LeBBZVoWHNZmbjfjU5" } }


In [6]:
# Perform a keyword search
keyword = "paper"
search_results, project_ids = search_index(index, keyword)


2025-01-14 22:19:41.920 | INFO     | super_helper:search_index:269 - Starting keyword search...
2025-01-14 22:19:41.921 | INFO     | super_helper:search_index:270 - Keyword: 'paper'
2025-01-14 22:19:41.945 | INFO     | super_helper:with_logging_block:33 - 
2025-01-14 22:19:41.946 | INFO     | super_helper:with_logging_block:34 - STARTING BLOCK: Keyword Search
2025-01-14 22:19:41.948 | INFO     | super_helper:with_logging_block:35 - ==================================================
2025-01-14 22:19:41.949 | INFO     | super_helper:search_index:288 - Total search results found for keyword 'paper': 2
2025-01-14 22:19:41.950 | INFO     | super_helper:search_index:291 - Listing all search results:
2025-01-14 22:19:41.953 | INFO     | super_helper:search_index:295 - Result 1: Project ID: 55943@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-14 22:19:41.956 | INFO     | super_helper:search_index:295 - Result 2: Project ID: 76811@test, File CID: QmXh37WXcrLXkJX2cAPjbPdK

With Reusable Helper Function for Block Separation and Tracing

A helper function, with_logging_block, which accepts a block name, the code block to execute, and optional parameters for context-specific tracing.

Cleanly separates each logical block, improving readability, modularity, and error tracing.

Includes the output of the search results explicitly logged within the Keyword Search block. This will ensure all search results are printed for visibility and debugging purposes.

In [7]:
with with_logging_block("Processing Search Results", logger):
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        with with_logging_block(f"Processing Result for Project ID: {project_id or 'Unknown'}", logger):
            if not project_id or not file_cid or not metadata_cid:
                logger.error(f"Missing required data in result: {result_dict}")
                continue

            logger.info(f"File CID: {file_cid}")
            logger.info(f"Metadata CID: {metadata_cid}")

            # Fetch project details
            with with_logging_block("Fetching Project Details", logger):
                project_details = get_account_detail(project_id)
                if not project_details:
                    logger.error(f"No project details found for Project ID: {project_id}.")
                    continue
                logger.info(f"Fetched project details for {project_id}: {project_details}")

            # Parse blockchain data
            with with_logging_block("Parsing Blockchain Data", logger):
                try:
                    blockchain_data = json.loads(project_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding project details JSON for {project_id}: {e}")
                    continue

            # Validate file CID
            with with_logging_block("Validating File CID", logger):
                validation_result = fetch_project_details(file_cid, blockchain_data)
                logger.info(f"Validation Result for {project_id}: {validation_result}")
                if not validation_result["is_valid"]:
                    logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")
                    continue

            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")

            # Process project metadata
            if project_metadata_cid:
                with with_logging_block("Processing Project Metadata", logger):
                    project_metadata = download_json_from_ipfs(project_metadata_cid)
                    logger.info(f"Downloaded project metadata: {project_metadata}")

            # Process linked user details
            if linked_user:
                with with_logging_block(f"Processing Linked User: {linked_user}", logger):
                    user_details = get_account_detail(linked_user)
                    try:
                        user_details = json.loads(user_details)
                    except json.JSONDecodeError as e:
                        logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                        continue

                    user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid")
                    if user_json_ld_cid:
                        user_metadata = download_json_from_ipfs(user_json_ld_cid)
                        logger.info(f"Downloaded user metadata: {user_metadata}")
                    else:
                        logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")

            # Process metadata CID
            if metadata_cid:
                with with_logging_block("Processing Metadata CID", logger):
                    file_metadata = download_json_from_ipfs(metadata_cid)
                    file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
                    logger.info(f"Downloaded file metadata: {file_metadata}")

2025-01-14 22:19:42.001 | INFO     | super_helper:with_logging_block:33 - 
2025-01-14 22:19:42.003 | INFO     | super_helper:with_logging_block:34 - STARTING BLOCK: Processing Search Results
2025-01-14 22:19:42.004 | INFO     | super_helper:with_logging_block:35 - ==================================================
2025-01-14 22:19:42.005 | INFO     | super_helper:with_logging_block:33 - 
2025-01-14 22:19:42.006 | INFO     | super_helper:with_logging_block:34 - STARTING BLOCK: Processing Result for Project ID: 55943@test
2025-01-14 22:19:42.008 | INFO     | super_helper:with_logging_block:35 - ==================================================
2025-01-14 22:19:42.009 | INFO     | __main__:<module>:12 - File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
2025-01-14 22:19:42.011 | INFO     | __main__:<module>:13 - Metadata CID: QmcG2x9U1YcRTB4zMob5QgwZDJWJLohYF25zNEeNJubzQf
2025-01-14 22:19:42.012 | INFO     | super_helper:with_logging_block:33 - 
2025-01-14 22:19:42.014 | INFO     |